In [1]:
import numpy as np 
from numpy import *
from random import *
import matplotlib.pyplot as plt
from PIL import Image
from pylab import array
from pylab import plot
import cv2
import os
import math
import time

In [2]:
M=2117
refs = zeros((M, 4), dtype=float)
refs_gt = zeros((M, 4), dtype=float)
f_vertical= open('247_Vertical.txt', 'r+')
f_line = f_vertical.read()
Label = np.zeros((M,20,2),dtype=np.float32)
PCA_Label = np.zeros((M,20,2),dtype=np.float32)
Res = np.zeros((M,20,2),dtype=np.float32)
PCA_Res = np.zeros((M,20,2),dtype=np.float32)

In [3]:
def PCA(dataMat,topNfeat=5):
    meanVals = np.mean(dataMat,axis = 0)
    dataMean = dataMat - meanVals
    conMat = dataMean.T.dot(dataMean)
    eigVals,eigVects = np.linalg.eig(conMat)   
    eigValInd = np.argsort(eigVals)
    eigValInd = eigValInd[:-(topNfeat+1):-1]
    redeigVects=eigVects[:,eigValInd] 
    lowdataMat = dataMean.dot(redeigVects)
    condata = (lowdataMat.dot(redeigVects.T)) + meanVals
    reducedata=lowdataMat+np.mean(dataMat)
    return condata

In [4]:
def get_gt_from_npy(file):
    for root, dirs, files in os.walk(file):
        for index,f in enumerate(files):
            print(index,f[0:16])
            if (not f.endswith('.npy')):
                continue       
            lab=np.load(os.path.join(root, f))
            Label[index]=lab
            pca_lab=PCA(lab,1)
            PCA_Label[index]=pca_lab

            if f[0:16] in f_line:
                ll = pca_lab[0][0],pca_lab[0][1],pca_lab[19][0],pca_lab[19][1]
                ll_x = [ll[0],ll[2]]
                ll_y = [ll[1],ll[3]]
                slope, intercept = np.polyfit(ll_x,ll_y,1)
                ll=(ll_y[0]-intercept)/slope,ll_y[0],(ll_y[1]-intercept)/slope,ll_y[1]
            else:
                ll = pca_lab[0][0],pca_lab[0][1],pca_lab[19][0],pca_lab[19][1]
                ll_x = [ll[0],ll[2]]
                ll_y = [ll[1],ll[3]]
                slope, intercept = np.polyfit(ll_x,ll_y,1)
                ll=ll_x[0],ll_x[0]*slope+intercept,ll_x[1],ll_x[1]*slope+intercept
            print(ll)
            refs_gt[index:] = ll
    return refs_gt,Label

In [5]:
file_dir=".\\dataWRSD\\annotations"
label_refs,Label = get_gt_from_npy(file_dir)
print(label_refs)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [6]:
def get_result_from_npy(file):
    for root, dirs, files in os.walk(file):
        for index,f in enumerate(files):
            print(index,f,f[8:24])
            if (not f.endswith('.npy')):
                continue       
            res=np.load(os.path.join(root, f))
            if f[8:24] in f_line:
                res = res[np.argsort(res[:,1])]
            else:
                res= res[np.argsort(res[:,0])]
            Res[index]=res
            pca_res=PCA(res,1)
            PCA_Res[index]=pca_res
            if f[8:24] in f_line:
                ll = pca_res[0][0],pca_res[0][1],pca_res[19][0],pca_res[19][1]
                ll_x = [ll[0],ll[2]]
                ll_y = [ll[1],ll[3]]
                slope, intercept = np.polyfit(ll_x,ll_y,1)
                print(ll_x,ll_y,slope, intercept)
                ll=(ll_y[0]-intercept)/slope,ll_y[0],(ll_y[1]-intercept)/slope,ll_y[1]

            else:
                ll = pca_res[0][0],pca_res[0][1],pca_res[19][0],pca_res[19][1]
                ll_x = [ll[0],ll[2]]
                ll_y = [ll[1],ll[3]]
                if ll_y[1]-ll_y[0]==0:
                    ll = ll_x[0],ll_y[0],ll_x[1],ll_y[1]
                else:
                    slope, intercept = np.polyfit(ll_x,ll_y,1)
                    ll=ll_x[0],ll_x[0]*slope+intercept,ll_x[1],ll_x[1]*slope+intercept

            print(ll)
            refs[index:] = ll
    return refs,Res

In [7]:
result_dir=".\\results\\coordsresult"
detected_refs,Res = get_result_from_npy(result_dir)
print(detected_refs)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [8]:
vec = zeros((M,2), dtype=float)
det = zeros((M,2), dtype=float)
angle_all= []
dist_all= []
totcnt=0
sum_angle=0
sum_dist=0
R=0.025*300
theta=math.pi/60
print('threshold:theta',math.degrees(theta),'threshold:distance',R)
for i in range(M):
    cenrefs_x=(label_refs[i][0]+label_refs[i][2])/2
    cenrefs_y=(label_refs[i][1]+label_refs[i][3])/2
    detrefs_x=(detected_refs[i][0]+detected_refs[i][2])/2
    detrefs_y=(detected_refs[i][1]+detected_refs[i][3])/2
    vec[i][0]=label_refs[i][2]-label_refs[i][0]
    vec[i][1]=label_refs[i][3]-label_refs[i][1]
    det[i][0]=detected_refs[i][2]-detected_refs[i][0]
    det[i][1]=detected_refs[i][3]-detected_refs[i][1]
    cosangle=(vec[i].dot(det[i]))/np.sqrt(vec[i].dot(vec[i]))/np.sqrt(det[i].dot(det[i]))

    rad_angle=math.acos(cosangle)
    angle=math.degrees(rad_angle)
    dist_mid=np.sqrt(pow(abs(cenrefs_x-detrefs_x),2)+pow(abs(cenrefs_y-detrefs_y),2))
    angle_all.append(angle)
    dist_all.append(dist_mid)
    if R > dist_mid:
        if theta > math.acos(cosangle):
            totcnt+=1

mean_angle=np.mean(angle_all)
var_angle=np.var(angle_all)
std_angle=np.std(angle_all)
mean_dist=np.mean(dist_all)
var_dist=np.var(dist_all)
std_dist=np.std(dist_all)
print('angle：Mean',mean_angle,'Var.',var_angle,'Std.',std_angle)
print('distance：Mean',mean_dist,'Var.',var_dist,'Std.',std_dist)
print(totcnt)
print('True Positive Rate:',totcnt/M)

threshold:theta 3.0 threshold:distance 7.5
angle：Mean nan Var. nan Std. nan
distance：Mean 0.0 Var. 0.0 Std. 0.0
0
True Positive Rate: 0.0


<ipython-input-8-e453014e34b9>:20: RuntimeWarning: invalid value encountered in double_scalars
  cosangle=(vec[i].dot(det[i]))/np.sqrt(vec[i].dot(vec[i]))/np.sqrt(det[i].dot(det[i]))


In [ ]:
def visualization(file_pathname,Lab,Det):
    for i,filename in enumerate(os.listdir(file_pathname)):
        print(i)
        mask = plt.imread(file_pathname+'\\'+filename)
        lab_x = Lab[i].T[0]
        lab_y = Lab[i].T[1]
        det_x = Det[i].T[0]
        det_y = Det[i].T[1]
        plt.figure(figsize=(4,3),dpi=100)
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
        plt.margins(0,0)       

        plot(lab_x,lab_y,color='blue',marker='o', linestyle='-', linewidth=1, markersize=4)
        plot(det_x,det_y,color='yellow',marker='o', linestyle='-', linewidth=1, markersize=4)
        plt.imshow(mask)
        plt.savefig('.\\results\\visual'+"\\"+filename,pad_inches=0)
        plt.show()
        plt.clf()
        plt.close()

file_path='.\\data\\WRSD\\images'
visualization(file_path,Label,Res)